# Second Method of authentication

## Short version (with session)

Import required python modules.

In [ ]:
import requests
import os

For the short version we are using the [Session](https://requests.readthedocs.io/en/master/user/advanced/#session-objects) functionality of the python requests module. Using a Session object will persist cookies across all requests.


1. We start by creating a Session instance.  
2. Open(GET request) the login page using the session instance this will persist the response cookies in the session
3. Create dictionary with required params to login
  * username/password: using os.environ.get to read the value from the "Enviroment Variables"
  * Set remember_me to true
  * _csrf: here we re-use the CSRF/XSRF token that was returned by the previous request  
4. Do the POST request with the data as [url params](https://requests.readthedocs.io/en/master/user/quickstart/#passing-parameters-in-urls)

In [ ]:
s = requests.Session()
s.get('https://profile.callofduty.com/cod/login')
data = {'username': os.environ.get('COD_EMAIL'), 
        'password': os.environ.get('COD_PW'), 
        'remember_me': 'true', 
        '_csrf': s.cookies['XSRF-TOKEN']}
s.post('https://profile.callofduty.com/do_login?new_SiteId=cod', params=data)

Test if the login worked by requesting private profile stats to see if we're properly authenticated.

In [ ]:
resp_profile = s.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/uno/uno/14688938695022220470/profile/type/warzone')
resp_profile.json()

## Longer Version (without session)

We get the login page and save the XSRF-TOKEN that is returned by the response and can be found in the response object. CSRF/XSRF tokens are used to prevent CSRF attacks: https://en.wikipedia.org/wiki/Cross-site_request_forgery

In [ ]:
get_login = requests.get('https://profile.callofduty.com/cod/login')
xsrf_token = get_login.cookies['XSRF-TOKEN']

Now that we have the XSRF token we can login and get the SSO tokens en ATKN tokens to do requests to private endpoints.
1. Create dictionary with required params to login
    * username/password: using os.environ.get to read the value from the "Enviroment Variables"
    * Set remember_me to true
    * _csrf: here we re-use the CSRF/XSRF token that was returned by the previous request
2. Do the POST request with the data as url params AND also pass the cookies that you got from the previous request it will contain a cookie that has the XSRF-TOKEN.


In [ ]:
data = {
    'username': os.environ.get('COD_EMAIL'), 
    'password': os.environ.get('COD_PW'), 
    'remember_me': 'true', 
    '_csrf': xsrf_token
}

cookies = {
    'XSRF-TOKEN':  xsrf_token,
}

post_login = requests.post('https://profile.callofduty.com/do_login?new_SiteId=cod', params=data, cookies=get_login.cookies)

Now we would expect the SSO tokens and ATKN to be in the response Cookies, unfortunately this is not case because in the response we are storing in "post_login" we've already been redirected twice and the post_login instance cookies won't contain the tokens.

In [ ]:
try: 
    post_login.cookies['ACT_SSO_COOKIE']
    print("ACT_SSO_COOKIE found")
except KeyError:
    print("ACT_SSO_COOKIE not found")

If you check the history of the response, you'll see there have been two previous response objects with status_code 302(redirect), if you check the cookies of the first response you'll find the tokens that we need.

In [ ]:
post_login.history

If we check the first Response instance at index zero of the list you'll see that it contains the tokens we need.

In [ ]:
try: 
    post_login.history[0].cookies['ACT_SSO_COOKIE']
    print("ACT_SSO_COOKIE found")
except KeyError:
    print("ACT_SSO_COOKIE not found")

post_login.history[0].cookies

As we only need the tokens of first response we could disable redirection in the post request adding the parameter allow_redirects=False

In [ ]:
post_login_wo_redirect = requests.post('https://profile.callofduty.com/do_login?new_SiteId=cod', params=data, cookies=get_login.cookies, allow_redirects=False)

Test if the login worked by requesting private profile stats to see if we're properly authenticated.  
We pass the cookies that were returned by the first response instance of the "do_login" call. These cookies contain the ACT_SSO_COOKIE & atkn values that are required by callofduty to prove that our client is authenticated to make this API call.
The cookies also contain a ACT_SSO_COOKIE_EXPIRY value that will indicate when the token will expirty and we'll have to renew it or authenticate again.

In [ ]:
resp_profile = s.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/uno/uno/14688938695022220470/profile/type/warzone', cookies=post_login_wo_redirect.cookies)
resp_profile.json()